In [2]:
# import 필요한 라이브러리
import os
from natsort import natsorted
from pathlib import Path
import shutil
import pandas as pd

# 환자 ID 대문자로 바꾸기
- ID의 일관성은 매우 중요함!
- 항상 모든 파일의 양식이 아래와 같은지 확인하기

fileName: P002_031220_w_0001_nonleg_imu_knee_angle_moment.csv

meaning:  patientID_visitDate_speed_Numbering_measuredLeg_imu_knee_angle_moment.csv

In [ ]:
# 환자 ID를 모두 대문자로 바꾸기
targetDir = "Z:\PROJECTS\iwalqq\Data\V3D\Output\IMU Deep Learning\Data\CSV\exported_csv"
for i in natsorted(os.listdir(targetDir)):
    old_name = os.path.join(targetDir, i)
    new_name = os.path.join(targetDir, 'P'+i[1:])
    os.rename(old_name,new_name)

# txt To csv 변환
- 필수적임
- 항상 데이터 받으면 수행할 것


In [ ]:
# txt to csv 변환
txtDir = r"Z:\PROJECTS\iwalqq\Data\V3D\Output\IMU Deep Learning\Data\20220325_raw_byDeepak"
csvDir = r"Z:\PROJECTS\iwalqq\Data\V3D\Output\IMU Deep Learning\Data\20220325_raw_byDeepak_csv"

dataList = natsorted([_ for _ in os.listdir(txtDir)])

for Name_datum in dataList:
    read_file = pd.read_csv(os.path.join(txtDir, Name_datum),sep='\t')
    read_file.to_csv(os.path.join(csvDir, Name_datum.replace(".txt",".csv")))
    

# txt To CSV 변환 시 oaleg의 side를 맨 마지막에 집어넣기

In [47]:
# txt to csv 변환
txtDir = r"Z:\PROJECTS\iwalqq\Data\V3D\Output\IMU Deep Learning\Data\20220325_raw_byDeepak"
csvDir = r"Z:\PROJECTS\iwalqq\Data\V3D\Output\IMU Deep Learning\Data\allnew_20220325_raw_byDeepak_csv"
dgDir = r'Z:\PROJECTS\iwalqq\Data\V3D\Output\IMU Deep Learning\demographics.xlsx'
dataList = natsorted([_ for _ in os.listdir(txtDir)])

# demographic 관련
list_demograph = pd.read_excel(dgDir,engine = 'openpyxl')

for Name_datum in dataList:
    sideInfo = list_demograph.loc[list_demograph['ID']==Name_datum.split('_')[0],'Side']
    read_file = pd.read_csv(os.path.join(txtDir, Name_datum),sep='\t')
    filename =  Name_datum.replace(".txt",".csv")
    filename = '.'.join([filename.split('.')[0]+f'_{sideInfo.values[0]}',filename.split('.')[1]])
    read_file.to_csv(os.path.join(csvDir,filename))

In [46]:
sideInfo.values[0]

'L'

# 하나의 폴더에 있는 csv 폴더별로 저장하기
- 현재는 필요없음

In [82]:
# 필요한 함수 선언
def ensure_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)

# target directory 지정
dataDir = r"Z:\PROJECTS\iwalqq\Data\V3D\Output\IMU Deep Learning\Data\CSV"
sortedFolder = "sorted_csv"
dataExt = r".csv"
# 파일 목록 전체 들고오고 필요한 확장자만 고르고 들고온 파일 목록에서 .txt만 남기고 나머지 것들 제외시키기
dataList = natsorted([_ for _ in os.listdir(os.path.join(dataDir,"exported_csv" )) if _.endswith(dataExt)])
countForErr = 0
# 파일 명 split하기
# 분리해서 각각 폴더로 옮기기
for datum in dataList:
    try:
        sep_datum = datum.split("_")
        paName = sep_datum[0]
        paVisitDate = sep_datum[1]
        paAffectedside = sep_datum[2]
        paDataside = sep_datum[3]
        paSpeed = sep_datum[4]
        #필요한 경로
        saveDir = os.path.join(dataDir, sortedFolder, paName, paVisitDate, paDataside, paSpeed)
        # 경로 확인
        ensure_dir(saveDir)
        # 파일 이동
        if not os.path.exists(os.path.join(saveDir,datum)): # 파일이 이미 있으면 그냥 지나감
            shutil.copyfile(os.path.join(dataDir,"exported_csv", datum), os.path.join(saveDir,datum))
    except IndexError:
        countForErr = countForErr + 1
        print(f'pass the weird file:{countForErr}')
      

#  folder로 분류된 data를 하나로 합치기(혹시나)
- 현재는 필요없음

In [68]:
# 위 과정을 거친 폴더를 다시 하나의 파일에다 저장하기
# 해당 폴더 내의 모든 .txt 파일 찾기
dataDir = r"Z:\PROJECTS\iwalqq\Data\V3D\Output\IMU Deep Learning\Data\sorted_csv"
exportFolder = r"Z:\PROJECTS\iwalqq\Data\V3D\Output\IMU Deep Learning\Data\exported_csv"
for (path, dir, files) in os.walk(dataDir):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.csv':
            shutil.copyfile(os.path.join(path, filename), os.path.join(exportFolder, filename))

# 제외시킬 데이터 일단은 COLUMN 이상한거 걸러내기


In [ ]:
# Column의 길이가 짧은 경우
# 다음 코드에서 이어서...